# EC1B1 Group project


In [279]:
# Downloading the data

import requests
import pandas as pd
from io import StringIO

# Google Sheets details
SHEET_ID = "1VnzB2MO5jEBzXqtkA0i_oNtONzOIg9y3I5nU1fB6DwU"
# List of sheets and their GIDs (Find these manually or automate if needed)
SHEET_GIDS = {
    "Sheet1": "0",  
    "Sheet2": "1811506178",
    "Sheet3": "1443003215",
    "Sheet4" : "746766865",
    "Sheet5" : "791351243",
    "Sheet6" : "68385636"
    
}

dfs = {}

# Loop through each sheet and fetch data
for sheet_name, gid in SHEET_GIDS.items():
    url = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={gid}"
    response = requests.get(url)

    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))  # Convert CSV text to DataFrame
        dfs[sheet_name] = df
        print(f"Extracted: {sheet_name} ✅")
    else:
        print(f"Failed to extract {sheet_name}, Status Code: {response.status_code}")

# Display extracted DataFrames
#print("Extracted Sheets:", dfs.keys())

# Example: Print first few rows of a sheet
#print(dfs["Sheet2"].head())  # Change "Sheet1" to the actual sheet name

germany_data=dfs["Sheet5"]
us_data=dfs["Sheet6"]
data=dfs['Sheet4']


Extracted: Sheet1 ✅
Extracted: Sheet2 ✅
Extracted: Sheet3 ✅
Extracted: Sheet4 ✅
Extracted: Sheet5 ✅
Extracted: Sheet6 ✅


In [280]:
us_data

,Unnamed: 0,Unnamed: 1,Dec 1959,Jan 1960,Feb 1960,Mar 1960,Apr 1960,May 1960,Jun 1960,Jul 1960,...,Mar 1990,Apr 1990,May 1990,Jun 1990,Jul 1990,Aug 1990,Sep 1990,Oct 1990,Nov 1990,Dec 1990
0,United States,"International Reserves and Liquidity, Reserves...","21,504.50Â M","21,478.10Â M","21,395.70Â M","21,344.70Â M","21,278.00Â M","21,234.30Â M","21,169.80Â M","20,999.40Â M",...,"76,303.03Â M","76,283.21Â M","77,028.37Â M","77,298.84Â M","77,906.29Â M","78,908.84Â M","80,024.17Â M","82,852.20Â M","83,059.40Â M","83,316.21Â M"
1,NaN,"Prices, Consumer Price Index, All items, Index",13.48,13.44,13.48,13.48,13.53,13.53,13.57,13.57,...,59.02,59.11,59.25,59.57,59.8,60.35,60.86,61.22,61.36,61.36


In [281]:

# df = us_data.drop(columns='USA')

# df.loc[-1] = df.columns  # Insert the column names as a row at index -1
# df.index = df.index + 1  # Shift index to start from 1 to maintain the order
# df = df.sort_index()  # Sort the DataFrame by index
# df.columns = range(df.shape[1])
# newdf = df.T
# # df.columns = df.iloc[0]

# newdf.columns = newdf.iloc[0]

# newdf = newdf.drop(0)

# newdf["Country"] = "USA"

# us_df = newdf

In [282]:
us_data#.drop(columns='Unnamed: 0	')
#germany_df = 
us_data.drop(us_data.columns[0], axis=1)
df5=us_data
df5.loc[-1] = df5.columns  # Insert the column names as a row at index -1
df5.index = df5.index + 1  # Shift index to start from 1 to maintain the order
df5 = df5.sort_index()  # Sort the DataFrame by index
df5.columns = range(df5.shape[1])
newdf5 = df5.T


newdf5 = newdf5.drop([0,1])





# germany_df.drop([0, 1])

newdf5.columns = newdf5.iloc[0]

#newdf4 = newdf4.drop(newdf4.index[1])

# newdf4.reset_index
# newdf5

newdf5.columns = [ 'Date', 'Reserves', 'CPI']

In [284]:
us_df = newdf5.reset_index(drop=True)


In [240]:
# # cont processing data as a whole

# df = data
# # Convert column headers into a row
# df = pd.DataFrame(df.values, columns=["Header"] * len(df.columns))

# # Add the original column names as a new row
# df.loc[-1] = df.columns.tolist()

# # Reset index to make the new row the first row
# df = df.reset_index(drop=True)

# # Print the result
# df
# # cont processing data as a whole

# data.T


In [241]:
germany_data#.drop(columns='Unnamed: 0	')
#germany_df = 
germany_data.drop(germany_data.columns[0], axis=1)
df4=germany_data
df4.loc[-1] = df4.columns  # Insert the column names as a row at index -1
df4.index = df4.index + 1  # Shift index to start from 1 to maintain the order
df4 = df4.sort_index()  # Sort the DataFrame by index
df4.columns = range(df4.shape[1])
newdf4 = df4.T


newdf4 = newdf4.drop([0,1])
# germany_df.drop([0, 1])

newdf4.columns = newdf4.iloc[0]

#newdf4 = newdf4.drop(newdf4.index[1])

# newdf4.reset_index

newdf4.columns = [ 'Date', 'Industrial Production', 'Exchange Rate', 'Reserves', 'CPI']


In [242]:
germany_df = newdf4.reset_index(drop=True)


In [243]:
# df3 = germany_data.drop(columns='Germany')

# df3.loc[-1] = df3.columns  # Insert the column names as a row at index -1
# df3.index = df3.index + 1  # Shift index to start from 1 to maintain the order
# df3 = df3.sort_index()  # Sort the DataFrame by index
# df3.columns = range(df3.shape[1])
# newdf3 = df3.T
# # df.columns = df.iloc[0]

# newdf3.columns = newdf3.iloc[0]

# newdf3 = newdf3.drop(0)

# newdf3["Country"] = "Germany"

# germany_df = newdf3




# POTENTIALLY DOESNT WORK I NEED TO CHECK IT 

In [244]:
# germany_df = germany_df.rename(columns={
#     'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar': 'Reserves', 
#     'Prices, Consumer Price Index, All items, Index': 'CPI',
#     'Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate': 'Exchange Rate',
#     'Economic Activity, Industrial Production, Index': 'Economic Activity' })

# DONT THINK I NEED THIS ANYMORE

# us_df = us_df.rename(columns={
#     'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar': 'Reserves', 
#     'Prices, Consumer Price Index, All items, Index': 'CPI', })



In [285]:
germany_df = germany_df.applymap(lambda x: str(x).replace('Â', '') if isinstance(x, str) else x)
us_df = us_df.applymap(lambda x: str(x).replace('Â', '') if isinstance(x, str) else x)


/tmp/ipykernel_365/4220658101.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  germany_df = germany_df.applymap(lambda x: str(x).replace('Â', '') if isinstance(x, str) else x)
/tmp/ipykernel_365/4220658101.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  us_df = us_df.applymap(lambda x: str(x).replace('Â', '') if isinstance(x, str) else x)


In [286]:
# processing germany data

import re


#might not need this function anymore
def clean_string(input_string):
    # Use regex to remove anything that's not a digit or a decimal point
    return re.sub(r'[^0-9.]', '', input_string)



def convert_to_int(value):
    if isinstance(value, str) and 'M' in value:
        value = value.replace(',', '')  # Remove commas
        return int(float(value.replace('M', '').strip()) * 1_000_000)
    return value  # Return as is if not in expected format

# Apply the function
germany_df['Reserves'] = germany_df['Reserves'].apply(convert_to_int)

us_df['Reserves'] = us_df['Reserves'].apply(convert_to_int)
#print(us_df.columns)

germany_df


us_df

,Date,Reserves,CPI
0,Dec 1959,21504500000,13.48
1,Jan 1960,21478100000,13.44
2,Feb 1960,21395700000,13.48
3,Mar 1960,21344700000,13.48
4,Apr 1960,21278000000,13.53
...,...,...,...
368,Aug 1990,78908840000,60.35
369,Sep 1990,80024170000,60.86
370,Oct 1990,82852200000,61.22
371,Nov 1990,83059400000,61.36


data has been processed into us_df anf germany_df with the cleaned data

In [287]:
germany_df.to_csv('germany_clean_data.csv', index=False)
us_df.to_csv('us_clean_data.csv', index=False)

In [ ]:
# calculating the monthly growth rate in nominal exchange rate

# # exchange rate growth = rate [1] - rate [0] / rate [0] *100

# for i in range (len(germany_df)-1):
#     exchange_rate_growth = ((germany_df['Exchage Rate'][i+1]-germany_df['Exchage Rate'][i])/germany_df['Exchage Rate'][i])*100
    

In [288]:
us_df.columns

Index(['Date', 'Reserves', 'CPI'], dtype='object')

In [251]:
germany_df = germany_df.reset_index(drop=True)  # Ensures a proper integer index

germany_df['Exchange Rate'] = germany_df['Exchange Rate'].str.replace(',', '').astype(float)


In [252]:
# CREATE EXCHANGE RATE GROWTH COLUMN

# Initialize a new column with NaN values
germany_df['Exchange Rate Growth'] = None

# Iteratively calculate and insert exchange rate growth
for i in range(1, len(germany_df)):  # Start from 1 because there's no previous value for the first row
    germany_df.loc[i, 'Exchange Rate Growth'] = ((germany_df.loc[i, 'Exchange Rate'] - germany_df.loc[i-1, 'Exchange Rate']) 
                                         / germany_df.loc[i-1, 'Exchange Rate']) * 100

# Display the updated DataFrame
germany_df

,Date,Industrial Production,Exchange Rate,Reserves,CPI,Exchange Rate Growth
0,Dec 1959,32.5,4.20,4811470000,24.62,None
1,Jan 1960,31.19,4.20,4724160000,24.62,0.0
2,Feb 1960,31.04,4.20,4806360000,24.48,0.0
3,Mar 1960,32.2,4.20,4966460000,24.48,0.0
4,Apr 1960,34.29,4.20,5236120000,24.62,0.0
...,...,...,...,...,...,...
368,Aug 1990,75.96,1.57,72425740000,67.56,-4.268293
369,Sep 1990,86.5,1.57,73197570000,67.77,0.0
370,Oct 1990,92.64,1.52,75011930000,68.26,-3.184713
371,Nov 1990,89.46,1.49,76166070000,68.12,-1.973684


In [253]:
# CODE FOR REAL EXCHANGE RATE AND REAL EXCHANGE RATE GROWTH

import pandas as pd

# Ensure numeric conversion
germany_df['Exchange Rate'] = pd.to_numeric(germany_df['Exchange Rate'], errors='coerce')
germany_df['CPI'] = pd.to_numeric(germany_df['CPI'], errors='coerce')
us_df['CPI'] = pd.to_numeric(us_df['CPI'], errors='coerce')

# Align US CPI data to the same time period as germany_df (assuming same dates in order)
germany_df['US CPI'] = us_df['CPI'].values  # Ensure correct alignment

# Calculate Real Exchange Rate (RER)
germany_df['Real Exchange Rate'] = germany_df['Exchange Rate'] * (germany_df['US CPI'] / germany_df['CPI'])

# Compute Monthly Growth in Real Exchange Rate
germany_df['Real Exchange Rate Growth'] = germany_df['Real Exchange Rate'].pct_change() * 100

# Display the updated DataFrame
germany_df

ValueError: Length of values (13) does not match length of index (373)

In [ ]:
# CODE FOR REAL EXCHANGE RATE INDEX

#dec_1990_rer = germany_df.loc[germany_df['Date'] == '1990-12', 'Real Exchange Rate'].values[0]


dec_1990_rer=germany_df['Real Exchange Rate'][0]
# Compute Real Exchange Rate Index
germany_df['Real Exchange Rate Index'] = germany_df['Real Exchange Rate'] / dec_1990_rer

# Display the updated DataFrame
germany_df

In [ ]:
# CODE FOR INFLATION

import pandas as pd


# Calculate monthly inflation rate
germany_df['Monthly Inflation Rate'] = germany_df['CPI'].pct_change() * 100

# Display the updated DataFrame
germany_df

In [ ]:
#CODE FOR INDUSTRIAL PRODUCTION GROWTH

import pandas as pd

# Ensure Industrial Production values are numeric
germany_df['Economic Activity'] = pd.to_numeric(germany_df['Economic Activity'], errors='coerce')

# Calculate monthly growth rate in industrial production
germany_df['Monthly Industrial Production Growth'] = germany_df['Economic Activity'].pct_change() * 100

# Display the updated DataFrame
print(germany_df)
